In [106]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
import json
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import RobertaModel, RobertaTokenizer


In [107]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [108]:
# load data
import pandas as pd
# data = [{'name': 'vikash', 'age': 27}, {'name': 'Satyam', 'age': 14}]
# test_data = pd.DataFrame.from_dict(data, orient='columns')
test_data = pd.read_json('data/test.jsonl', lines=True)
dev_data = pd.read_json('data/dev.jsonl', lines=True)
train_data = pd.read_json('data/train.jsonl', lines=True)
test_data.head(10)

,premise,choice1,choice2,question,label,idx
0,The man turned on the faucet.,The toilet filled with water.,Water flowed from the spout.,effect,1,0
1,The girl found a bug in her cereal.,She poured milk in the bowl.,She lost her appetite.,effect,1,1
2,The woman retired.,She received her pension.,She paid off her mortgage.,effect,0,2
3,I wanted to conserve energy.,I swept the floor in the unoccupied room.,I shut off the light in the unoccupied room.,effect,1,3
4,The hamburger meat browned.,The cook froze it.,The cook grilled it.,cause,1,4
5,I doubted the salesman's pitch.,I turned his offer down.,He persuaded me to buy the product.,effect,0,5
6,I decided to stay home for the night.,The forecast called for storms.,My friends urged me to go out.,cause,0,6
7,My eyes became red and puffy.,I was sobbing.,I was laughing.,cause,0,7
8,The flame on the candle went out.,I blew on the wick.,I put a match to the wick.,cause,0,8
9,The man drank heavily at the party.,He had a headache the next day.,He had a runny nose the next day.,effect,0,9


In [109]:
# test
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# test_sequence = "{" + "effect" + "}" + "I ran the ice cube under warm water."
test_sequence = "{"+ test_data.iloc[0]['question'] + "}" + test_data.iloc[0]['premise']
print("test_sequence is: ", test_sequence)
print(tokenizer(test_sequence))
print(tokenizer.tokenize(test_sequence))

test_sequence is:  {effect}The man turned on the faucet.
{'input_ids': [0, 45152, 26715, 24303, 133, 313, 1224, 15, 5, 856, 17527, 594, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['{', 'effect', '}', 'The', 'Ġman', 'Ġturned', 'Ġon', 'Ġthe', 'Ġf', 'auc', 'et', '.']


In [110]:
# test 2
test_sequence = "{"+ test_data.iloc[1]['question'] + "}" + test_data.iloc[1]['premise']
print("test_sequence is: ", test_sequence)
print(tokenizer(test_sequence))
print(tokenizer.tokenize(test_sequence))

test_sequence is:  {effect}The girl found a bug in her cereal.
{'input_ids': [0, 45152, 26715, 24303, 133, 1816, 303, 10, 13673, 11, 69, 25629, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['{', 'effect', '}', 'The', 'Ġgirl', 'Ġfound', 'Ġa', 'Ġbug', 'Ġin', 'Ġher', 'Ġcereal', '.']


In [111]:
print(test_data.shape[0])

100


## Model Construction

In [119]:
# Model_1
from torch import nn
from transformers import RobertaConfig, RobertaModel

class OurRobertaCOPA(torch.nn.Module):
    def __init__(self):
        super(OurRobertaCOPA, self).__init__()
        # self.configuration = RobertaConfig()
        # self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        # self.l1 = RobertaModel(self.configuration)
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.l1.requires_grad = True
        self.softmax = nn.Softmax(dim=0)
        self.pre_classifier = torch.nn.Linear(768, 512)
        self.dropout = torch.nn.Dropout(0.3)
        # self.classifier = torch.nn.Linear(768, 5)
        # hidden_dim=32 for later trials.
        # self.lstm = nn.LSTM(768, 32, 1, bias=False)
        self.output_layer = nn.Linear(512, 2)

    def forward(self, sequence_1, sequence_2):
        # Two input here
        token_1 = tokenizer(sequence_1)
        token_2 = tokenizer(sequence_2)
        output_1 = self.l1(input_ids=torch.tensor(token_1["input_ids"]).unsqueeze(0), attention_mask=torch.tensor(token_1["attention_mask"]).unsqueeze(0))
        output_2 = self.l1(input_ids=torch.tensor(token_2["input_ids"]).unsqueeze(0), attention_mask=torch.tensor(token_2["attention_mask"]).unsqueeze(0))
        # RobertaModel(RobertaConfig())

        # _, (hidden_rep_1, _) = self.lstm(output_1.unsqueeze(0))
        # _, (hidden_rep_2, _) = self.lstm(output_2.unsqueeze(0))
        # _, (hidden_rep_1, _) = self.lstm(output_1)
        # _, (hidden_rep_2, _) = self.lstm(output_2)

        hidden_rep_1 = torch.nn.ReLU()(self.pre_classifier(output_1[0])).squeeze(0)
        hidden_rep_2 = torch.nn.ReLU()(self.pre_classifier(output_2[0])).squeeze(0)
        # hidden_rep_1 = self.pre_classifier(output_1[0]).squeeze(0)
        # hidden_rep_2 = self.pre_classifier(output_2[0]).squeeze(0)
        # print("-------hidden_rep_1:")
        # print(hidden_rep_1)
        # print(hidden_rep_1.size())
        # print("-------hidden_rep_2:")
        # print(hidden_rep_2)
        # print(hidden_rep_2.size())
        
        # hidden_rep = torch.cat((hidden_rep_1.unsqueeze(1), hidden_rep_2.unsqueeze(1)), 1)
        hidden_rep = self.dropout(torch.cat((hidden_rep_1, hidden_rep_2), 0))

        # print("-------hidden_rep:")
        # print(hidden_rep)
        # print(hidden_rep.size())

        output = self.output_layer(hidden_rep.unsqueeze(0))
        # print("-------output:")
        # print(output)
        # print(output.size())
        # print("--------------")

        output_squezzed = output.squeeze(0).squeeze(0)
        # print("-------output_squezzed:")
        # print(output_squezzed)
        # print(output_squezzed.size())
        # print("--------------")
        
        # y_hat = softmax(output_squezzed)
        # y_sum =  torch.sum(y_hat, 0)
        # col1= torch.sum(y_hat, 0)[0]
        # col2 = torch.sum(y_hat, 0)[1]
        # y_result = torch.tensor(torch.argmax(y_sum)).type(torch.FloatTensor)
        # y_result = torch.tensor(y_sum)
        
        return output_squezzed

In [113]:
# Model_3, use only the very last hidden layer from Roberta.
from torch import nn
from transformers import RobertaConfig, RobertaModel

class OurRobertaCOPA_last_hr(torch.nn.Module):
    def __init__(self):
        super(OurRobertaCOPA_last_hr, self).__init__()
        # self.configuration = RobertaConfig()
        # self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        # self.l1 = RobertaModel(self.configuration)
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.l1.requires_grad = True
        self.softmax = nn.Softmax(dim=0)
        self.pre_classifier = torch.nn.Linear(768, 512)
        self.dropout = torch.nn.Dropout(0.3)
        # self.classifier = torch.nn.Linear(768, 5)
        # hidden_dim=32 for later trials.
        # self.lstm = nn.LSTM(768, 32, 1, bias=False)
        self.output_layer = nn.Linear(1024, 2)

    def forward(self, sequence_1, sequence_2):
        # Two input here
        token_1 = tokenizer(sequence_1)
        token_2 = tokenizer(sequence_2)
        output_1 = self.l1(input_ids=torch.tensor(token_1["input_ids"]).unsqueeze(0), attention_mask=torch.tensor(token_1["attention_mask"]).unsqueeze(0))[-1]
        output_2 = self.l1(input_ids=torch.tensor(token_2["input_ids"]).unsqueeze(0), attention_mask=torch.tensor(token_2["attention_mask"]).unsqueeze(0))[-1]
        # RobertaModel(RobertaConfig())

        # _, (hidden_rep_1, _) = self.lstm(output_1.unsqueeze(0))
        # _, (hidden_rep_2, _) = self.lstm(output_2.unsqueeze(0))
        # _, (hidden_rep_1, _) = self.lstm(output_1)
        # _, (hidden_rep_2, _) = self.lstm(output_2)

        hidden_rep_1 = torch.nn.ReLU()(self.pre_classifier(output_1[0])).squeeze(0)
        hidden_rep_2 = torch.nn.ReLU()(self.pre_classifier(output_2[0])).squeeze(0)
        # hidden_rep_1 = self.pre_classifier(output_1[0]).squeeze(0)
        # hidden_rep_2 = self.pre_classifier(output_2[0]).squeeze(0)
        # print("-------hidden_rep_1:")
        # print(hidden_rep_1)
        # print(hidden_rep_1.size())
        # print("-------hidden_rep_2:")
        # print(hidden_rep_2)
        # print(hidden_rep_2.size())
        
        # hidden_rep = torch.cat((hidden_rep_1.unsqueeze(1), hidden_rep_2.unsqueeze(1)), 1)
        hidden_rep = self.dropout(torch.cat((hidden_rep_1, hidden_rep_2), 0))

        # print("-------hidden_rep:")
        # print(hidden_rep)
        # print(hidden_rep.size())

        output = self.output_layer(hidden_rep.unsqueeze(0))
        # print("-------output:")
        # print(output)
        # print(output.size())
        # print("--------------")

        output_squezzed = output.squeeze(0).squeeze(0)
        # print("-------output_squezzed:")
        # print(output_squezzed)
        # print(output_squezzed.size())
        # print("--------------")
        
        # y_hat = softmax(output_squezzed)
        # y_sum =  torch.sum(y_hat, 0)
        # col1= torch.sum(y_hat, 0)[0]
        # col2 = torch.sum(y_hat, 0)[1]
        # y_result = torch.tensor(torch.argmax(y_sum)).type(torch.FloatTensor)
        # y_result = torch.tensor(y_sum)
        
        return output_squezzed

In [114]:
# model_2
class COPAClassifier(nn.Module):
    def __init__(self, output_size=1):
        super().__init__()
        
        # default pretrained layer
        self.roberta = RobertaModel.from_pretrained('roberta-base')
#         self.roberta.requires_grad = False
        self.classifier = nn.Linear(768, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.classifier(self.roberta(input_ids=x)[1])
        
    def predict(self, x):
        return 1 if self.sigmoid(self.forward(x)) > 0.5 else 0

## Training

In [120]:
# Model_1
model = OurRobertaCOPA()
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OurRobertaCOPA(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,),

In [121]:
model_2 = COPAClassifier()
model_2.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


COPAClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((7

In [122]:
model_3 = OurRobertaCOPA_last_hr()
model_3.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OurRobertaCOPA_last_hr(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [124]:
# model_1, use all the hidden reps. Sum the scores later.
epochs = 10
ce = nn.CrossEntropyLoss()
softmax = nn.Softmax(dim=0)
optimizer = optim.SGD(model.parameters(), lr=0.05)

for j in range(epochs):
    print('### Epoch: ' + str(j+1) + ' ###')
    av_loss = 0
    model.train()
    for i in range(0, train_data.shape[0]):

        x1 = "{"+ train_data.iloc[i]['question'] + "}" + train_data.iloc[i]['premise'] + train_data.iloc[i]['choice1']
        x2 = "{"+ train_data.iloc[i]['question'] + "}" + train_data.iloc[i]['premise'] + train_data.iloc[i]['choice2']
        # x1 = "{"+ train_data.iloc[i]['question'] + "}" + train_data.iloc[i]['premise'] 
        # x2 = train_data.iloc[i]['choice1'] + train_data.iloc[i]['choice2']
        y = torch.tensor(train_data.iloc[i]['label'])

        y_raw = model(x1, x2)

        # y_hat = softmax(y_raw)
        y_sum =  torch.sum(y_raw, 0)
        # col1= torch.sum(y_hat, 0)[0]
        # col2 = torch.sum(y_hat, 0)[1]
        # y_result = torch.tensor(torch.argmax(y_sum)).type(torch.FloatTensor)
        y_result = torch.tensor(y_sum, requires_grad = True)
        # print("y_result = ", y_result.unsqueeze(0).shape)
        # print("y = ", y.unsqueeze(0).shape)
        # print("y_result.unsqueeze(0): ", y_result.unsqueeze(0))
        # print("y.unsqueeze(0): ", y.unsqueeze(0))
        loss = ce(y_result.unsqueeze(0), y.unsqueeze(0))
        av_loss += loss

        # print("loss: ", loss)
        loss.backward()

        optimizer.step()
    print("av_loss: ", av_loss / train_data.shape[0])

### Epoch: 1 ###


C:\Users\light\AppData\Local\Temp/ipykernel_29112/365599480.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_result = torch.tensor(y_sum, requires_grad = True)


av_loss:  tensor(1.6144, grad_fn=<DivBackward0>)
### Epoch: 2 ###
av_loss:  tensor(1.5883, grad_fn=<DivBackward0>)
### Epoch: 3 ###
av_loss:  tensor(1.5948, grad_fn=<DivBackward0>)
### Epoch: 4 ###
av_loss:  tensor(1.6655, grad_fn=<DivBackward0>)
### Epoch: 5 ###
av_loss:  tensor(1.6065, grad_fn=<DivBackward0>)
### Epoch: 6 ###
av_loss:  tensor(1.5880, grad_fn=<DivBackward0>)
### Epoch: 7 ###
av_loss:  tensor(1.6456, grad_fn=<DivBackward0>)
### Epoch: 8 ###
av_loss:  tensor(1.6020, grad_fn=<DivBackward0>)
### Epoch: 9 ###
av_loss:  tensor(1.6601, grad_fn=<DivBackward0>)
### Epoch: 10 ###
av_loss:  tensor(1.6245, grad_fn=<DivBackward0>)


In [105]:
# model_3, use only the very last hidden rep
epochs = 10
ce = nn.CrossEntropyLoss()
softmax = nn.Softmax(dim=0)
optimizer = optim.SGD(model.parameters(), lr=0.1)

for j in range(epochs):
    print('### Epoch: ' + str(j+1) + ' ###')
    av_loss = 0
    model.train()
    for i in range(0, train_data.shape[0]):

        # x1 = "{"+ train_data.iloc[i]['question'] + "}" + train_data.iloc[i]['premise'] + train_data.iloc[i]['choice1']
        # x2 = "{"+ train_data.iloc[i]['question'] + "}" + train_data.iloc[i]['premise'] + train_data.iloc[i]['choice2']
        x1 = "{"+ train_data.iloc[i]['question'] + "}" + train_data.iloc[i]['premise'] 
        x2 = train_data.iloc[i]['choice1'] + train_data.iloc[i]['choice2']
        y = torch.tensor(train_data.iloc[i]['label'])


        y_raw = model_3(x1, x2)

        # y_hat = softmax(y_raw)
        # y_sum =  torch.sum(y_raw, 0)
        # col1= torch.sum(y_hat, 0)[0]
        # col2 = torch.sum(y_hat, 0)[1]
        # y_result = torch.tensor(torch.argmax(y_sum)).type(torch.FloatTensor)
        y_result = torch.tensor(y_raw, requires_grad = True)
        # print("y_result = ", y_result.unsqueeze(0).shape)
        # print("y = ", y.unsqueeze(0).shape)
        # print("y_result.unsqueeze(0): ", y_result.unsqueeze(0))
        # print("y.unsqueeze(0): ", y.unsqueeze(0))
        loss = ce(y_result.unsqueeze(0), y.unsqueeze(0))
        av_loss += loss

        # print("loss: ", loss)
        loss.backward()

        optimizer.step()
    print("av_loss: ", av_loss / train_data.shape[0])


### Epoch: 1 ###


C:\Users\light\AppData\Local\Temp/ipykernel_29112/4075531021.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_result = torch.tensor(y_raw, requires_grad = True)


av_loss:  tensor(0.6948, grad_fn=<DivBackward0>)
### Epoch: 2 ###
av_loss:  tensor(0.6943, grad_fn=<DivBackward0>)
### Epoch: 3 ###
av_loss:  tensor(0.6949, grad_fn=<DivBackward0>)
### Epoch: 4 ###
av_loss:  tensor(0.6971, grad_fn=<DivBackward0>)
### Epoch: 5 ###
av_loss:  tensor(0.6949, grad_fn=<DivBackward0>)
### Epoch: 6 ###
av_loss:  tensor(0.6961, grad_fn=<DivBackward0>)
### Epoch: 7 ###
av_loss:  tensor(0.6971, grad_fn=<DivBackward0>)
### Epoch: 8 ###
av_loss:  tensor(0.6945, grad_fn=<DivBackward0>)
### Epoch: 9 ###
av_loss:  tensor(0.6958, grad_fn=<DivBackward0>)
### Epoch: 10 ###
av_loss:  tensor(0.6959, grad_fn=<DivBackward0>)


## Testing

In [78]:
# Use trained model 1 to predict and calculate the accuracy
correct = 0
for i in range(0, dev_data.shape[0]):
    x1 = "{"+ dev_data.iloc[i]['question'] + "}" + dev_data.iloc[i]['premise'] + dev_data.iloc[i]['choice1']
    x2 = "{"+ dev_data.iloc[i]['question'] + "}" + dev_data.iloc[i]['premise'] + dev_data.iloc[i]['choice2']
    y = torch.tensor(dev_data.iloc[i]['label'])
    y_raw = model(x1, x2)
    y_sum = torch.sum(y_raw, 0)
    print("y_sum: ", y_sum)
    y_result = torch.argmax(torch.tensor(y_sum))
    print("y_result: ", y_result)
    if y_result == y:
        correct += 1

acc = correct / len(train_data)
print(acc)

y_sum:  tensor([ -2.7796, -10.4463], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([ 0.9028, -7.2679], grad_fn=<SumBackward1>)
y_result:  tensor(0)


C:\Users\light\AppData\Local\Temp/ipykernel_29112/977614026.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_result = torch.argmax(torch.tensor(y_sum))


y_sum:  tensor([ -0.8715, -11.7578], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([-0.2110, -7.1787], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([-0.8815, -8.9295], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([ 0.5134, -6.4749], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([-0.0863, -8.9692], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([-1.5716, -7.3077], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([ 0.1018, -5.7898], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([ 0.0418, -9.5596], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([ 0.0270, -8.0548], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([-1.1956, -8.4085], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([-1.5862, -9.2693], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([-0.3193, -6.4562], grad_fn=<SumBackward1>)
y_result:  tensor(0)
y_sum:  tensor([ 0.3352, -7.7915], gra

KeyboardInterrupt: 

In [ ]:
from datetime import datetime

def load_data(filename):
    raw_data = pd.read_json(filename, lines=True)
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    dataset = []
    
    for i, row in raw_data.iterrows(): # yeah, yeah, I know...
        prem = torch.tensor(tokenizer(row['premise'])['input_ids'])
        h1 = torch.tensor(tokenizer(row['choice1'])['input_ids'])
        h2 = torch.tensor(tokenizer(row['choice2'])['input_ids'])
        dataset.append((torch.cat((prem, h1, h2)), torch.tensor(row['label']).to(torch.float32)))
        
    return dataset

train_data = load_data('COPA/train.jsonl')
print(f'Training data loaded (length {len(train_data)})')
dev_data = load_data('COPA/dev.jsonl')
print(f'Dev data loaded (length {len(dev_data)})')
test_data = load_data('COPA/test.jsonl')
print(f'Test data loaded (length {len(test_data)})')

# 3) Now we train our model. 
start_time = datetime.now()
epochs = 1
bce = nn.BCELoss()
sigmoid = nn.Sigmoid()
optimizer = optim.Adam(model_2.parameters(), lr=0.05)

for i in range(epochs):
    print('### Epoch: ' + str(i+1) + ' ###')
    av_loss = 0
    model_2.train()
    for (x, y) in train_data:
        optimizer.zero_grad()
        
        # a) calculate probs / get an output
        out = model_2(x.unsqueeze(0))
        y_hat = sigmoid(out.squeeze(0).squeeze(0))
        
        # b) compute loss
        loss = bce(y_hat, y)
        av_loss += loss
        
        # c) get the gradient
        loss.backward()

        # d) update the weights
        optimizer.step()
    print(av_loss/len(train_data))

end_time = datetime.now()
print(f'Training completed in {str(end_time - start_time)}')

In [19]:
# Just some random tests
a = torch.randn(4, 4)
print(a)

print(torch.sum(a, 0))


tensor([[-0.6343,  0.9115, -0.9311, -0.2760],
        [-1.1090, -2.3171, -1.4745,  1.7214],
        [ 1.0259, -1.1309,  0.2277, -0.3401],
        [-0.6796, -1.1034, -1.7867,  0.5659]])
tensor([-1.3970, -3.6399, -3.9647,  1.6712])
